### Import necessary libraries

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

### Load dataframe

In [3]:
df = pd.read_csv("./data/bus.csv", sep = ';', index_col = 0)
#df

### Define objectives

In [4]:
### DEFAULT: all set to 'max'

### Method 1 - by a list
objectives = ['max', 'max', 'min', 'max', 'min', 'min', 'min', 'max']

### Method 2 - by a word
objectives = 'max'

### Method 3 - by a dictionary
objectives = {
    "MaxSpeed" : 'max',
    "ComprPressure" : "max",
    "Blacking" : "min",
    "Torque" : "max",
    "SummerCons" : "min",
    "WinterCons" : "min",
    "OilCons" : "min",
    "HorsePower" : "max"}

### By default all objectives are set to 'max'. Synonyms to 'max': "gain", 'g'. Synonyms to 'min': 'cost', 'c'.

'''
###TODO Method 4 - providing in a dictionary objectives for not all criteria eg.:
    
    objectives = {
        "SummerCons" : "min",
        "WinterCons" : "min"
    }

    The rest by default are set to 'max'

    Justification: case in which we have 20 criteria, but only one is a cost type criterion

###TODO Method 5 - option to change default 'max' to default 'min' + Method 4:

    Justification: case in which we have 20 criteria, but only one is a gain type criterion

'''

'\n###TODO Method 4 - providing in a dictionary objectives for not all criteria eg.:\n    \n    objectives = {\n        "SummerCons" : "min",\n        "WinterCons" : "min"\n    }\n\n    The rest by default are set to \'max\'\n\n    Justification: case in which we have 20 criteria, but only one is a cost type criterion\n\n###TODO Method 5 - option to change default \'max\' to default \'min\' + Method 4:\n\n    Justification: case in which we have 20 criteria, but only one is a gain type criterion\n\n'

### Define weights

In [5]:
### DEFAULT: all set to 1

### Method 1 - by a list
weights = [2, 5, 7, 8, 6.5, 1, 1, 3]

### Method 2 - by a dictionary
weights = {
    "MaxSpeed" : 2,
    "ComprPressure" : 5,
    "Blacking" : 7,
    "Torque" : 8,
    "SummerCons" : 6.5,
    "WinterCons" : 1,
    "OilCons" : 1,
    "HorsePower" : 3}

### Define your own range of values, by default minimal and maximal values are taken for each objective

In [6]:
### DEFAULT: lower_bound(criterion) = min(criterion), upper_bound(criterion) = max(criterion)

### Method 1 - by a 2D list
expert_range = [[1, 10], [1, 10], [0, 10], [-5, 5], [0, 10], [0, 10], [0, 100], [1, 2]] #TODO with this line bug in improvement_features()

### Method 2 - by a dictionary

expert_range = {
    "MaxSpeed" : [1, 10],
    "ComprPressure" : [1, 10],
    "Blacking" : [0, 10],
    "Torque" : [-5, 5],
    "SummerCons" : [0, 10],
    "WinterCons" : [0, 10],
    "OilCons" : [0, 100],
    "HorsePower" : [1, 2]}


### Method 3 - by a 1D list
expert_range = [1, 6]

'''
###TODO Method 4 - providing in a dictionary expert_ranges for not all criteria eg.:
    
    expert_range = {
        "MaxSpeed" : [1, 10],
        "ComprPressure" : [1, 10]
    }

    The rest by default are set to [min(criterion), max(criterion)]

'''


'\n###TODO Method 4 - providing in a dictionary expert_ranges for not all criteria eg.:\n    \n    expert_range = {\n        "MaxSpeed" : [1, 10],\n        "ComprPressure" : [1, 10]\n    }\n\n    The rest by default are set to [min(criterion), max(criterion)]\n\n'

### Create buses object

In [7]:
from src import MSDTransformer as msdt

In [8]:
# buses = msdt.MSDTransformer("A")
# buses = msdt.MSDTransformer("I")
# buses = msdt.MSDTransformer("R")
# buses = msdt.MSDTransformer(msdt.ATOPSIS)
# buses = msdt.MSDTransformer(msdt.ITOPSIS)
buses = msdt.MSDTransformer(msdt.RTOPSIS)

### Before performing transformation to ranking, fit the data

In [9]:
buses.fit(df, weights, objectives,  expert_range)

### Transform data to ranking

In [11]:
buses.transform(None)
''' 
TODO no need to call both fit and transform, instead call fit_transform(df, weights, objectives,  expert_range)
'''

print(buses.ranked_alternatives)
''' 
TODO show ranked_alternatives in a dataframe form, probably by using dedicated method
    
    buses.show_ranked_alternatives( start_rank = 1, last_rank = 10, mode = 'minimal')

    All parameters optional: 
        start_rank and last_rank allow to show only a part of ranking in case we don't want to see 100 positions,
        mode allow to show additional columns like sd, mean, aggregation value, etc.
'''

['b08', 'b15', 'b19', 'b23', 'b14', 'b10', 'b17', 'b20', 'b11', 'b29', 'b13', 'b27', 'b03', 'b07', 'b12', 'b02', 'b06', 'b31', 'b24', 'b30', 'b26', 'b09', 'b05', 'b18', 'b21', 'b16', 'b32', 'b25', 'b28', 'b22', 'b01', 'b04']


" \nTODO show ranked_alternatives in a dataframe form, probably by using dedicated method\n    \n    buses.show_ranked_alternatives( start_rank = 1, last_rank = 10, mode = 'minimal')\n\n    All parameters optional: \n        start_rank and last_rank allow to show only a part of ranking in case we don't want to see 100 positions,\n        mode allow to show additional columns like sd, mean, aggregation value, etc.\n"

### Check how you should change a mean of objectives for one alternative, to improve its position in the ranking

In [12]:
alternative_to_improve = buses.data.loc["b02"].copy()
alternative_to_improve_CS = buses.original_data.loc["b02"].copy()
alternative_to_overcome = buses.data.loc["b09"].copy()
improvement_ratio = 0.01
weights_mean = np.mean(buses.weights)

buses.agg_fn.improvement_mean(alternative_to_improve, alternative_to_overcome, improvement_ratio, weights_mean)

'''
###TODO alternative_to_improve:
    METHOD 1: by its name ("b02")
    METHOD 2: by its id (2)
    METHOD 3: by a list ([10, 1, 1, 1, 1, 3.5, 6, 1000]) #Currently only this method has a support

###TODO alternative_to_overcome:
    METHOD 1: by its name ("b09")
    METHOD 2: by its id (9)

###TODO weights_mean
    Get rid of this one.
'''

It is impossible to improve with only mean


'\n###TODO alternative_to_improve:\n    METHOD 1: by its name ("b02")\n    METHOD 2: by its id (2)\n    METHOD 3: by a list ([10, 1, 1, 1, 1, 3.5, 6, 1000]) #Currently only this method has a support\n\n###TODO alternative_to_overcome:\n    METHOD 1: by its name ("b09")\n    METHOD 2: by its id (9)\n\n###TODO weights_mean\n    Get rid of this one.\n'

### Check how you should change a sd of objectives for one alternative, to improve its position in the ranking

In [13]:
alternative_to_improve = buses.data.loc[buses.ranked_alternatives[27]].copy()
alternative_to_overcome = buses.data.loc[buses.ranked_alternatives[22]].copy()
improvement_ratio = 0.01
weights_mean = np.mean(buses.weights)

buses.agg_fn.improvement_std(alternative_to_improve, alternative_to_overcome, improvement_ratio, weights_mean)

'''
###TODO alternative_to_improve:
    METHOD 1: by its name ("b02")
    METHOD 2: by its id (2)
    METHOD 3: by a list ([10, 1, 1, 1, 1, 3.5, 6, 1000]) #Currently only this method has a support

###TODO alternative_to_overcome:
    METHOD 1: by its name ("b09")
    METHOD 2: by its id (9)

###TODO weights_mean
    Get rid of this one.
'''

You should change standard deviation by  12.083355003484131


'\n###TODO alternative_to_improve:\n    METHOD 1: by its name ("b02")\n    METHOD 2: by its id (2)\n    METHOD 3: by a list ([10, 1, 1, 1, 1, 3.5, 6, 1000]) #Currently only this method has a support\n\n###TODO alternative_to_overcome:\n    METHOD 1: by its name ("b09")\n    METHOD 2: by its id (9)\n\n###TODO weights_mean\n    Get rid of this one.\n'

### Check how you should change particular values of features for one alternative, to improve its position in the ranking

In [14]:
alternative_to_improve = buses.data.loc[buses.ranked_alternatives[27]].copy()
alternative_to_overcome = buses.data.loc[buses.ranked_alternatives[22]].copy()
improvement_ratio = 0.01
objectives_to_change = ['MaxSpeed', 'Blacking', 'SummerCons']

buses.agg_fn.improvement_features(alternative_to_improve, alternative_to_overcome, improvement_ratio, buses.weights, objectives_to_change, buses.value_range, buses.objectives)

'''
###TODO alternative_to_improve:
    METHOD 1: by its name ("b02")
    METHOD 2: by its id (2)
    METHOD 3: by a list ([10, 1, 1, 1, 1, 3.5, 6, 1000]) #Currently only this method has a support

###TODO alternative_to_overcome:
    METHOD 1: by its name ("b09")
    METHOD 2: by its id (9)

### TODO objectives_to_change
    METHOD 1: by a list (['MaxSpeed', 'Blacking', 'SummerCons']) #Currently only this method has a support
    METHOD 2: by a string ('MaxSpeed')

###TODO weights
    Get rid of this one.

###TODO value_range
    Get rid of this one.

###TODO objectives
    Get rid of this one.

###TODO Print in form of df.
'''

MaxSpeed        -84.0
ComprPressure     0.0
Blacking        -41.5
Torque            0.0
SummerCons        0.0
WinterCons        0.0
OilCons           0.0
HorsePower        0.0
Name: b25, dtype: float64


'\n###TODO alternative_to_improve:\n    METHOD 1: by its name ("b02")\n    METHOD 2: by its id (2)\n    METHOD 3: by a list ([10, 1, 1, 1, 1, 3.5, 6, 1000]) #Currently only this method has a support\n\n###TODO alternative_to_overcome:\n    METHOD 1: by its name ("b09")\n    METHOD 2: by its id (9)\n\n### TODO objectives_to_change\n    METHOD 1: by a list ([\'MaxSpeed\', \'Blacking\', \'SummerCons\']) #Currently only this method has a support\n    METHOD 2: by a string (\'MaxSpeed\')\n\n###TODO weights\n    Get rid of this one.\n\n###TODO value_range\n    Get rid of this one.\n\n###TODO objectives\n    Get rid of this one.\n\n###TODO Print in form of df.\n'

### Check how you should change value of a single feature for one alternative, to improve its position in the ranking

In [15]:
alternative_to_improve = buses.data.loc["b02"].copy()
alternative_to_overcome = buses.data.loc["b09"].copy()
improvement_ratio = 1e-8
feature_to_change = 'HorsePower'

buses.agg_fn.improvement_features(alternative_to_improve, alternative_to_overcome, improvement_ratio, buses.weights, [feature_to_change],
                                  buses.value_range, buses.objectives)

lower_bounds = buses.original_data.min().values.copy()
upper_bounds = buses.original_data.max().values.copy()
alternative_to_improve_CS = buses.original_data.loc["b02"].copy()
buses.agg_fn.improvement_single_feature(alternative_to_improve, alternative_to_overcome, improvement_ratio, buses.weights, feature_to_change, buses.value_range, buses.objectives, alternative_to_improve_CS, lower_bounds, upper_bounds)

'''
###TODO alternative_to_improve:
    METHOD 1: by its name ("b02")
    METHOD 2: by its id (2)
    METHOD 3: by a list ([10, 1, 1, 1, 1, 3.5, 6, 1000]) #Currently only this method has a support

###TODO alternative_to_overcome:
    METHOD 1: by its name ("b09")
    METHOD 2: by its id (9)

### TODO objectives_to_change
    METHOD 1: by a list (['MaxSpeed', 'Blacking', 'SummerCons']) #Currently only this method has a support
    METHOD 2: by a string ('MaxSpeed')

### TODO lower_bounds
    By default it should take a min value for the criterion

### TODO upper_bounds
    By default it should take a max value for the criterion
    
###TODO weights
    Get rid of this one.

###TODO value_range
    Get rid of this one.

###TODO objectives
    Get rid of this one.
'''

This set of features to change is not sufficient to overcame that alternative
Solutions: -65.05050289190407 314.51186735169637
Criterion HorsePower is gain type, we need to increase value
Neither solution is feasible
It is impossible to improve changing only HorsePower


'\n###TODO alternative_to_improve:\n    METHOD 1: by its name ("b02")\n    METHOD 2: by its id (2)\n    METHOD 3: by a list ([10, 1, 1, 1, 1, 3.5, 6, 1000]) #Currently only this method has a support\n\n###TODO alternative_to_overcome:\n    METHOD 1: by its name ("b09")\n    METHOD 2: by its id (9)\n\n### TODO objectives_to_change\n    METHOD 1: by a list ([\'MaxSpeed\', \'Blacking\', \'SummerCons\']) #Currently only this method has a support\n    METHOD 2: by a string (\'MaxSpeed\')\n\n### TODO lower_bounds\n    By default it should take a min value for the criterion\n\n### TODO upper_bounds\n    By default it should take a max value for the criterion\n    \n###TODO weights\n    Get rid of this one.\n\n###TODO value_range\n    Get rid of this one.\n\n###TODO objectives\n    Get rid of this one.\n'

### Check transformation of "new" alternatives

In [16]:
result1 = buses.data.iloc[:, -3:]
result2 = pd.DataFrame(buses.transform_new_data(df.values, normalize_data=True), index=["Mean", "Std", "AggFn"], columns=df.index).T
np.allclose(result1, result2)

True